In [2]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy
import pandas as pd
from keras.models import load_model
import streamlit as st
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle
import seaborn as sns
from sklearn.preprocessing import StandardScaler


In [38]:
df = pd.read_csv("nepse_data.csv")

In [39]:
df.dropna(inplace=True)

In [40]:
    # Sort the dataframe by date
    df = df.sort_values('Date')

In [46]:
df

,Symbol,Date,Open,High,Low,Close,Percent Change,Volume
1129,NEPSE,2019-01-27,1171.35,1171.72,1160.84,1160.84,0.00,"306,070,016.00"
1128,NEPSE,2019-01-28,1162.94,1164.89,1161.48,1164.89,0.00,"203,579,008.00"
1127,NEPSE,2019-01-29,1164.91,1165.09,1160.10,1160.99,0.00,"259,283,008.00"
1126,NEPSE,2019-01-30,1160.53,1163.27,1159.35,1160.59,0.00,"190,572,992.00"
1125,NEPSE,2019-01-31,1162.34,1163.86,1160.26,1161.63,0.00,"268,244,992.00"
...,...,...,...,...,...,...,...,...
4,NEPSE,2024-01-08,2025.89,2059.66,2025.62,2059.49,1.80,"4,675,224,906.28"
3,NEPSE,2024-01-09,2063.56,2098.04,2063.42,2080.12,1.00,"6,118,317,866.67"
2,NEPSE,2024-01-10,2084.59,2116.44,2084.44,2089.62,0.45,"5,134,848,789.38"
1,NEPSE,2024-01-11,2091.28,2106.43,2082.49,2088.99,-0.03,"4,307,763,277.27"


In [43]:
    # Convert '--' to 0 in the 'Percent Change' column
    df['Percent Change'] = df['Percent Change'].str.replace('--', '0')

In [44]:

    # Convert '--' to 0 in the 'Percent Change' column
    df['Percent Change'] = df['Percent Change'].str.replace('%', "")

In [45]:
    # Convert 'Percent Change' column to float
    df['Percent Change'] = df['Percent Change'].astype(float)

In [47]:
    # Create features and target variables
    X = df[['Open', 'High', 'Low', 'Percent Change']]
    y = df['Close']

In [48]:
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

In [49]:
    # Create an SVR model
    svr = SVR()


In [50]:
    # Fit the model
    svr.fit(X_train, y_train)

SVR()

In [51]:
    # Make predictions on the test set
    y_pred = svr.predict(X_test)

In [53]:
    # Calculate RMSE, MAE, and R2 for training set
    train_rmse = np.sqrt(mean_squared_error(y_train, svr.predict(X_train)))
    train_mae = mean_absolute_error(y_train, svr.predict(X_train))
    train_r2 = r2_score(y_train, svr.predict(X_train))

    # Calculate RMSE, MAE, and R2 for test set
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    test_mae = mean_absolute_error(y_test, y_pred)
    test_r2 = r2_score(y_test, y_pred)

    # Print evaluation metrics for training set
    print("Training RMSE:", train_rmse)
    print("Training MAE:", train_mae)
    print("Training R2:", train_r2)

    # Print evaluation metrics for test set
    print("Test RMSE:", test_rmse)
    print("Test MAE:", test_mae)
    print("Test R2:", test_r2)


Training RMSE: 395.0500558013131
Training MAE: 309.52157877514367
Training R2: 0.5075668609546591
Test RMSE: 419.90367689279196
Test MAE: 344.0844663535454
Test R2: 0.5034151097447597


In [59]:
    # Forecast close prices for the upcoming week
    last_day = df['Date'].max()
    
    forecast_features = df[['Open', 'High', 'Low', 'Percent Change']].tail(1).values


In [68]:
predictions = []
for _ in range(7):
    prediction = svr.predict(forecast_features)[0]
    predictions.append(prediction)
    forecast_features = np.roll(forecast_features, -1, axis=0)
    forecast_features[-1] = [df['Open'].iloc[-1], df['High'].iloc[-1], df['Low'].iloc[-1], predictions[-1]]

C:\Users\ramsi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\ramsi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\ramsi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\ramsi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\ramsi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Us

In [69]:
    # Create a DataFrame for the predictions
    df_predictions = pd.DataFrame(predictions, columns=['close_price'])


In [70]:
    # Print the dataframe
    print(df_predictions)

   close_price
0  1975.163194
1  1975.163194
2  1975.163194
3  1975.163194
4  1975.163194
5  1975.163194
6  1975.163194
